# DSCI 100 Group Project Report - Group 17

## Introduction

Introduction:
* provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
* clearly state the question you tried to answer with your project
* identify and describe the dataset that was used to answer the question



Auditing firms have a very important responsibility in catching fraud cases. Firms performing audits must carefully examine the financial records of businesses to ensure that they are following accounting laws. To properly perform audits, auditors must go through financial records such as receipts, balance sheets, etc. This is a lot of data to analyze. The question we want to answer is: can fruadulent firms can be correctly predicted using risk factors from the dataset and a k-nearest neighbours approach?

The dataset being used will show annual audit data from 777 firms across 14 different sectors. There are 17 columns of observations in the dataset, the risk class column shows whether the case is fraudulent or not. 

### Variable Information

**Note:** no units were provided for variables

| | Inherent Risk Factors | | Control Risk Factors |
| --- | --- | ---| --- |
| ParaA | Discrepancy in planned-expenditure of inspection and summary report A in Rs | Sector score | Historical risk score value of the target-unit |
| ParaB | Discrepancy in unplanned-expenditure of inspection and summary report B in Rs | Loss | Amount of loss suffered by the firm last year. |
| Total | Total amount of discrepancy found in other reports Rs | History | Average historical loss suffered by firm in the last 10 years. |
| Number | Historical discrepancy score. | District score | Historical risk score of a district in the last 10 years. |
| Money Value | Amount of money involved in misstatements in the past audits. | | |
| Sector ID | Unique ID of the target sector. | Location ID | Unique ID of the city/province. |
| ARS | Total risk score using analytical procedure. | Audit ID | Unique Id assigned to an audit case. |
| Risk class | Risk Class assigned to an audit-case, 1=fraud or 0=no fraud (Target Feature) 

## Methods and Results

Methods & Results:
* describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis. 
* Your report should include code which:
    * loads data from the original source on the web
    * wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysi
    * performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
    * creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
    * performs the data analysis
    * creates a visualization of the analysis 
    * note: all tables and figure should have a figure/table number and a legend



## Discussion

Discussion:

* summarize what you found
* discuss whether this is what you expected to find?
* discuss what impact could such findings have?
* discuss what future questions could this lead to?



## References

References:

* At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references)
* Make sure to cite the source of your data as well.



Hooda, Nishtha & Bawa, Seema & Rana, Prashant. (2018). Fraudulent Firm Classification: A Case Study of an External Audit. Applied Artificial Intelligence. 32. 10.1080/08839514.2018.1451032. 